In [1]:
%matplotlib notebook
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import glob

import MosaicFitsReader as mfr
import FindSources as fs

In [2]:
 def centroid(arr):
    """
    One step 1D centroiding algo.
    Returns centroid position and standard deviation
    """
    l = arr.shape[0]
    ixs = np.arange(l)
    arr = arr - np.median(arr)
    arr = np.where(arr < 0, 0, arr)    
    ixs2 = ixs * ixs
    sumarr = arr.sum()
    cen = np.dot(arr, ixs)/sumarr
    return cen, math.sqrt(np.dot(arr, ixs2)/sumarr - cen * cen)

def centroidLoop(arr, fromIdx, toIdx, nLoops=10, epsilon=1E-1):
    """
    Finds the centroid by repeatedly centering and recalculating 
    until the centroid position changes by less than epsilon.
    
    Returns status, centroid position, standard deviation, iterations
    
    status: 0 OK, -1 bad centroid or no signal
    centroid position: position relative to input array, ie. 0 is first pixel
    standard deviation: standard deviation as calculated by the centroid algorithm (assumed Gaussian stats)
    iterations: number of iterations needed until change is less than epsilon
    """
    def limit(x):
        if x < 0: return 0
        if x >= length: return length
        return x
    
    length = len(arr)
    radius = (toIdx - fromIdx)/2
    lastCenPos = -9999
    for i in range(nLoops):
        fromIdx = int(limit(fromIdx))
        toIdx = int(limit(fromIdx + radius + radius + 0.5))
        pos, cenStd = centroid(arr[fromIdx:toIdx])
        cenPos = pos + fromIdx
        #print (i, fromIdx, toIdx, cenPos, cenStd, lastCenPos)
        
        if cenPos < fromIdx or toIdx < cenPos:
            return -1, 0, 0, i
        
        if abs(lastCenPos - cenPos) < epsilon:
            return 0, cenPos, cenStd, i
        if cenStd > radius/3:
            return -1, cenPos, cenStd, i
        fromIdx = cenPos - radius
        lastCenPos = cenPos
        
    return -1, cenPos, cenStd, i  

In [ ]:
def findWidths (arr1d, size=60):
    """
    Divides the input array in segments of size length.
    For each segment, finds the centroid, if centroid is good then record it.
    Sorts the centroids by standard deviation.
    Returns the smallest half of the standard deviation
    """
    out = []
    for x in range(0, len(arr1d)-size, size):
        ok, cen, std, idx = centroidLoop(arr1d, x, x+size)     
        #print (res)
        if ok == 0:
            out.append((std, cen))
    #print (out)
    if len(out) <= 0:
        return []
    out = sorted(out, key=lambda x: x[0])
    return out[:len(out)//2]

def findFWHMs (img, gridSize=90):
    fc = fs.FindSources(img, minContrast=0)
    cents = fc.findAll(gridSize, 0.8)
    cents = [x for x in cents if x[2] < 10] #x[2] is fwhm
    return cents

In [ ]:
def measureFocusWidthSpec (prefix, fromNr, toNr, focusKeyword='BLUFOCUS', column=1700, segmSize=60):
    """
    For all input files, finds the standard deviations of the centroids.
    These standard deviations are assosicated with the focus. 

    Output is stored in out[].
    """

    out = []
    for fnr in range(fromNr, toNr):
        fname = "%s%04d.fits" % (prefix, fnr)
        ffile = mfr.MosaicFitsReader(fname)
        img = ffile.data
        focus = ffile.getKeyword(focusKeyword)
        if focus == None:
            continue
        cut1d = img[:,column]
        widths = findWidths(cut1d, segmSize)
        widthsT = np.array(widths).T
        out.extend(( (focus,x) for x in widthsT[0] ))
    return np.array(out).T

def measureFocusWidthImg (prefix, fromNr, toNr, focusKeyword='BLUFOCUS', gridSize=90):
    """
    For all input files, finds the standard deviations of the centroids.
    These standard deviations are assosicated with the focus. 

    Output is stored in out[].
    """

    out = []
    for fnr in range(fromNr, toNr):
        fname = "%s%04d.fits" % (prefix, fnr)
        ffile = mfr.MosaicFitsReader(fname)
        img = ffile.data
        focus = ffile.getKeyword(focusKeyword)
        if focus == None:
            continue
        cents = findFWHMs(img, gridSize)
        centsT = np.array(cents).T
        out.extend(( (focus,x) for x in centsT[2] ))
    return np.array(out).T

600 -0.75751066 2.87576475943 2.55172746274 3.03778340778 0.0810093241744
700 -0.75751066 2.84977280307 2.6521593591 2.94857952506 0.0494033609932
800 -0.75751066 2.84059971163 2.60925175423 2.95627369033 0.0578369893508
900 -0.75751066 2.95150595567 2.78762488262 3.03344649219 0.0409702682602
1000 -0.75751066 2.95224060136 2.7541201014 3.05130085135 0.0495301249921
1100 -0.75751066 2.97890071893 2.7830864258 3.0768078655 0.0489535732829
1200 -0.75751066 2.95650518147 2.74851066647 3.06050243897 0.0519986287491
1300 -0.75751066 2.78521109009 2.46051391814 2.94755967607 0.0811742929873
1400 -0.75751066 2.95184204928 2.7875151722 3.03400548782 0.0410817192711
1500 -0.75751066 2.9810411794 2.40515035941 3.26898658939 0.143972704998
1600 -0.75751066 3.06382567844 2.27089556976 3.46029073278 0.198232527169
1700 -0.75751066 2.80981955613 2.72439879025 2.85252993907 0.0213551914706
1800 -0.75751066 2.86421956405 2.63282418616 2.979917253 0.0578488444722
1900 -0.75751066 2.86321841904 2.708846

600 -0.55736351 3.08963744111 2.75249320947 3.25820955693 0.084286057911
700 -0.55736351 2.99548158627 2.80171670835 3.09236402524 0.0484412194806
800 -0.55736351 2.93936638928 2.74136790827 3.03836562979 0.0494996202527
900 -0.55736351 2.86774277872 2.75373799927 2.92474516844 0.0285011948615
1000 -0.55736351 2.92030404498 2.62552667168 3.06769273164 0.0736943433265
1100 -0.55736351 2.8675693112 2.69753886462 2.95258453448 0.0425076116429
1200 -0.55736351 2.96856034915 2.65705542763 3.1243128099 0.0778762303782
1300 -0.55736351 3.22672402865 2.66041314583 3.50987947005 0.141577720705
1400 -0.55736351 2.87979641994 2.62693930474 3.00622497753 0.0632142787996
1500 -0.55736351 2.85526799792 2.58884715107 2.98847842134 0.0666052117123
1600 -0.55736351 3.02800604021 2.2581269489 3.41294558587 0.192469772827
1700 -0.55736351 2.95421617702 2.78078451715 3.04093200695 0.043357914968
1800 -0.55736351 3.18007745997 2.76814339725 3.38604449133 0.10298351568
1900 -0.55736351 3.15467931822 2.70837

In [ ]:
def findBestFocus(pairs):
    """
    Fits a hyperbola: x=focus, y=standard deviation

    Hyperbola equation: y^2 = Ax^2 + Bx + C

    """
    res = np.polyfit(pairs[0], np.multiply(pairs[1], pairs[1]), deg=2)

    func = np.poly1d(res)
    def func1 (x):
        return math.sqrt(func(x))

    funcV = np.vectorize(func1)
    A, B, C = res
    minX = -B/2/A
    return minX, funcV

In [ ]:

plotWidthVsFocus(rpairs, rminX, f2wFunc=rfuncV)

<IPython.core.display.Javascript object>